In [9]:
from IPython.core.display import display, HTML
%config IPCompleter.greedy = True
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [10]:
# Import libraries and define constants

import os
from datetime import datetime
from free_bwacs_model import FreeBWACS
from restricted_bwacs_model import RestrictedBWACS
from src.helpers import sorted_alphanumeric, save_results_on_file

RUNS = 10
INSTANCES_FOLDER = 'instances/CVRPLIB/CMT'
INSTANCES = sorted_alphanumeric([instance_name.replace(
    '.vrp', '') for instance_name in os.listdir(INSTANCES_FOLDER)])
# INSTANCES = ['CMT1']
SAVE_RESULTS = True
SAVE_IMGS = False
RESULTS_FILE_WITH_HEADER = True
BASE_RESULTS_FOLDER = 'results'
THIS_EXECUTION_FOLDER = datetime.now().strftime("%Y%m%d_%H%M%S")


In [11]:
# Execution of runs for instances files for FREE-ANT

instances_with_errors = []
cluster_type = 'kmeans'  # 'kmeans' or 'kmedoids'
ant_type = 'fa'  # 'fa', 'ra' or 'random'

# Validation of folder to save results
if SAVE_RESULTS:
    if not os.path.exists(BASE_RESULTS_FOLDER):
        os.makedirs(BASE_RESULTS_FOLDER)

# Start running the algorithm
for instance in INSTANCES:
    
    solution_list = []
    results_folder = f'{THIS_EXECUTION_FOLDER}/{instance}'
    file_name = f'{str(instance).lower()}_{ant_type}_{cluster_type}'

    if SAVE_IMGS: os.makedirs(f'{BASE_RESULTS_FOLDER}/{results_folder}/imgs', exist_ok=True)
    
    for run in range(RUNS):
        try:
            bwacs = FreeBWACS(
                instance=f'{INSTANCES_FOLDER}/{instance}',
                max_nodes=9999,
                cluster_type=cluster_type,
                metric='euclidian',
                tare_percentage=0.15,
                max_iterations=100,
                total_ant_divider=1,
                only_k_optimum=True,
                start_ant_on_best_nodes=True,
                # heuristic_type will let you choice what information gotta build the heuristic information matrix
                # 0 - Only distances data / Best beta: 5
                # 1 - Only energies data / Best gamma: 5
                # 2 - Distances data * energies data / Best parameters: 5 - 5
                # 3 - Only distances data of Savings algorithm / Best delta: 1
                # 4 - Only energies data of Savings algorithm / Best eta: 2
                # 5 - Distances data * distances savings data / Best parameters: 4 - 5
                # 6 - Energies data * energies savings data / Best parameters: 5 - 5
                # 7 - Distances data * energies data * distances savings data / Best parameters: 3 - 4 - 3
                # 8 - Distances data * energies data * energies savings data / Best parameters: 3 - 4 - 3
                # 9 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
                # 10 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
                # 11 - Distances data * energies data * distances savings data * energies savings data * capacity utilization data / Best parameters: 2 - 4 - 3 - 4 - 1
                heuristic_type=2,
                # Importance of pheromones trace. Recommended alpha value in: [1, 4]
                alpha=3,
                # Importance of distance heuristic information. Recommended beta value in: [2, 5]
                beta=2,
                # Importance of energy heuristic information. Recommended gamma value in: [2, 5]
                gamma=3,
                # Importance of distances's Savings Algorithm information. Recommended delta value in: [0, 2]
                delta=1,
                # Importance of energies's Savings Algorithm information. Recommended eta value in: [0, 2]
                eta=1,
                # Importance of vehicle's capacity utilization. Recommended mi value in: [0, 2]
                mi=0,
                # Define the strategy used for pheromone updating. - options: | 0 - 1 |
                # The value 0 is for the classic function in ACO: 1 / Best_Solution_Quality
                # The value 1 if for the improve function know as Ant Weight Strategy.
                pheromone_updating_strategy=0,
                local_ant_update_pheromones=False,
                best_iteration_ant_update_pheromones=False,
                best_global_ant_update_pheromones=True,
                penalize_worst_solution=True,
                mutate_pheromones_matrix=True,
                # Evaporation rate of the pheromones by the next function: phoromone =  (1 - p) * pheromone. Recommended value for p: 0.02.
                p=0.02,
                # Mutation probability for every row of the pheromones matrix, such as: mutate if (random number between 0 and 1) <= Pm
                # Recommended value for Pm: 0.3.
                Pm=0.3,
                # Mutation intensity for the pheromones matrix. Recommended value for sigma in: [2, 4]
                sigma=3,
                # Constant used in Pseudorandom Transition Rule. This is a probability of choice the next node by argmax, define in Ant Colony System.
                # Recommended value for l0 in: [0.2, 0.4]
                l0=0.3,
                # Constant used for a lot of functions, such as: f(.) = H / specific_values.
                H=1,
                ls_ant_solution=False,
                ls_best_iteration=True,
                ls_best_global=False,
                use_normalized_matrix=False,
                print_instance=False,
                print_clusters=False,
                print_solution=False,
                print_distance_matrix=False,
                print_energy_matrix=False,
                print_distance_saving_matrix=False,
                print_energy_saving_matrix=False,
                print_combination_matrix=False,
                print_pheromone_matrix=False,
                output_sol_img=f'{BASE_RESULTS_FOLDER}/{results_folder}/imgs/{file_name}.png'
            )

            solution_energy, solution_arcs, solution_distance, solution_time = bwacs.solve()
            solution_list.append(
                (solution_energy, solution_arcs, solution_distance, solution_time))
        except Exception as e:
            instances_with_errors.append((instance, e))
            break
    
    if (len(solution_list) > 0): 
        print(f'Solutions for instance {instance}: {solution_list}')
        print(f'{RUNS} runs for instance {instance} finished at {datetime.now().strftime("%Y%m%d_%H%M%S")}')
        print(
            f'Best solution for instance {instance}: {min(solution_list, key=lambda x: x[0])}')
        print(
            f'Worst solution for instance {instance}: {max(solution_list or [], key=lambda x: x[0])}')
        

        if SAVE_RESULTS:
            os.makedirs(
                f'{BASE_RESULTS_FOLDER}/{results_folder}', exist_ok=True)
            save_results_on_file(f'{BASE_RESULTS_FOLDER}/{results_folder}', file_name, ant_type, cluster_type,
                                solution_list, instance, 'csv', 'energy,arcs,distance,time(ms)')

if  len(instances_with_errors) > 0: 
    print(f'Instances with errors: {instances_with_errors}')
    os.makedirs(f'{BASE_RESULTS_FOLDER}/{results_folder}', exist_ok=True)
    error_file = open(f'{BASE_RESULTS_FOLDER}/{THIS_EXECUTION_FOLDER}/error_instances.txt', 'a')
    for instance, error in instances_with_errors:
        error_file.write(f'{instance}: {error}\n')
    error_file.close()


• INITIALIZING ALGORITHM •
------------------------------

• 0. Reading instance file
------------------------------

    - Parameters:
        > Instance name: instances/CVRPLIB/CMT/CMT1
        > Number of nodes: 51
        > Nodes and demands: {0: 0, 1: 7, 2: 30, 3: 16, 4: 9, 5: 21, 6: 15, 7: 19, 8: 23, 9: 11, 10: 5, 11: 19, 12: 29, 13: 23, 14: 21, 15: 10, 16: 15, 17: 3, 18: 41, 19: 9, 20: 28, 21: 8, 22: 8, 23: 16, 24: 10, 25: 28, 26: 7, 27: 15, 28: 14, 29: 6, 30: 19, 31: 11, 32: 12, 33: 23, 34: 26, 35: 17, 36: 6, 37: 9, 38: 15, 39: 14, 40: 7, 41: 27, 42: 13, 43: 11, 44: 16, 45: 10, 46: 5, 47: 25, 48: 17, 49: 18, 50: 10}
        > Total demand: 777 units
        > Vehicles capacity: 160 units
        > K-Optimum: 5
        > Tightness ratio: 0.97125
        > Total of iterations: 100
        > Number of ants per iteration: 50

• 1. Starting clustering process
------------------------------

    > CLUSTER TYPE: KMEANS
    > ITERACIÓN 1
        - Centroides actuales: [array([57, 58]),

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:438: RuntimeWarning: divide by zero encountered in true_divide
  1, self.distances_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:462: RuntimeWarning: divide by zero encountered in true_divide
  1, self.energies_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:568: RuntimeWarning: divide by zero encountered in true_divide
  self.np.divide(1, self.distances_matrix), self.beta)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:570: RuntimeWarning: divide by zero encountered in true_divide
  self.np.divide(1, self.energies_matrix), self.gamma)


        - Centroides actuales: [array([37., 69.]), array([48.6, 34.4]), array([19.88888889, 25.66666667]), array([35.8, 20.2]), array([19.2, 55.8])]
        - Nodos sin asignar: [], []
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 9867.190655216931

    > Mejor Clusterización: [array([ 2, 16, 29, 11, 22, 32, 20,  1,  3, 35], dtype=int64), array([ 9, 49, 30, 50, 10, 38, 34, 21, 39, 33, 45], dtype=int64), array([18, 13,  4, 41, 25, 14, 19], dtype=int64), array([17, 37, 12, 47, 44,  5, 15, 46, 42, 40], dtype=int64), array([48,  7, 23,  8, 26,  6, 27, 31, 24, 43], dtype=int64)], con costo final: 8066.528567941032, nodos sin asignar: [28 36]
    > Mejor Clusterización (Contraint): [array([ 2, 29, 20, 16, 21, 22,  3, 11, 35, 36], dtype=int64), array([49,  9, 38,  5, 10, 30, 50, 34, 39, 43], dtype=int64), array([18, 47, 12, 14, 46, 25, 24], dtype=int64), array([ 4, 17, 37, 44, 42, 41, 19, 15, 13, 45, 40, 33], dtype=int64), array([48,  8, 27, 23,  6,  7,  1, 2

KeyboardInterrupt: 

In [12]:
# Execution of runs for instances files for RESTRICTED-ANT

instances_with_errors = []
cluster_type = 'kmeans'  # 'kmeans' or 'kmedoids'
ant_type = 'ra'  # 'fa', 'ra' or 'random'

# Validation of folder to save results
if SAVE_RESULTS:
    if not os.path.exists(BASE_RESULTS_FOLDER):
        os.makedirs(BASE_RESULTS_FOLDER)

# Start running the algorithm
for instance in INSTANCES:

    solution_list = []
    results_folder = f'{THIS_EXECUTION_FOLDER}/{instance}'
    file_name = f'{str(instance).lower()}_{ant_type}_{cluster_type}'

    if SAVE_IMGS:
        os.makedirs(
            f'{BASE_RESULTS_FOLDER}/{results_folder}/imgs', exist_ok=True)

    for run in range(RUNS):
        try:
            bwacs = RestrictedBWACS(
                    instance=f'{INSTANCES_FOLDER}/{instance}',
                    max_nodes=999,
                    ant_type='emvrp',
                    cluster_type=cluster_type,
                    metric='euclidian',
                    tare=0.15,
                    max_iterations=50,
                    max_ants=50,
                    start_ant_on_best_nodes=1,
                    # Importance of pheromones trace. Recommended alpha value in: [1, 4]
                    alpha=2,
                    # Importance of distance heuristic information. Recommended beta value in: [2, 5]
                    beta=1,
                    # Importance of energy heuristic information. Recommended gamma value in: [2, 5]
                    gamma=3,
                    # Importance of distances's Savings Algorithm information. Recommended delta value in: [0, 2]
                    delta=1,
                    # Importance of energies's Savings Algorithm information. Recommended eta value in: [0, 2]
                    eta=1,
                    # Importance of vehicle's capacity utilization. Recommended mi value in: [0, 2]
                    mi=0,
                    pheromone_updating_strategy=0,
                    local_ant_update_pheromones=0,
                    best_iteration_update_pheromones=1,
                    best_global_update_pheromones=1,
                    penalize_worst_solution=1,
                    mutate_pheromones_matrix=1,
                    # Evaporation rate of the pheromones by the next function: phoromone =  (1 - p) * pheromone. Recommended value for p: 0.02.
                    p=0.02,
                    # Mutation probability for every row of the pheromones matrix, such as: mutate if (random number between 0 and 1) <= Pm
                    # Recommended value for Pm: 0.3.
                    Pm=0.3,
                    # Mutation intensity for the pheromones matrix. Recommended value for sigma in: [2, 4]
                    sigma=3,
                    # Constant used in Pseudorandom Transition Rule. This is a probability of choice the next node by argmax, define in Ant Colony System.
                    # Recommended value for l0 in: [0.2, 0.4]
                    q0=0.3,
                    # Constant used for a lot of functions, such as: f(.) = H / specific_values.
                    Q=1,
                    # heuristic_type will let you choice what information gotta build the heuristic information matrix
                    # 0 - Only distances data / Best beta: 5
                    # 1 - Only energies data / Best gamma: 5
                    # 2 - Distances data * energies data / Best parameters: 5 - 5
                    # 3 - Only distances data of Savings algorithm / Best delta: 1
                    # 4 - Only energies data of Savings algorithm / Best eta: 2
                    # 5 - Distances data * distances savings data / Best parameters: 4 - 5
                    # 6 - Energies data * energies savings data / Best parameters: 5 - 5
                    # 7 - Distances data * energies data * distances savings data / Best parameters: 3 - 4 - 3
                    # 8 - Distances data * energies data * energies savings data / Best parameters: 3 - 4 - 3
                    # 9 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
                    # 10 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
                    # 11 - Distances data * energies data * distances savings data * energies savings data * capacity utilization data / Best parameters: 2 - 4 - 3 - 4 - 1
                    heuristic_type=2,
                    ls_ant_solution=0,
                    ls_final_solution=0,
                    output_sol_img=f'{BASE_RESULTS_FOLDER}/{results_folder}/imgs/{file_name}.png'
            )

            solution_energy, solution_arcs, solution_distance, solution_time = bwacs.solve()
            solution_list.append(
                (solution_energy, solution_arcs, solution_distance, solution_time))
        except Exception as e:
            instances_with_errors.append((instance, e))
            break

    if (len(solution_list) > 0):
        print(f'Solutions for instance {instance}: {solution_list}')
        print(f'{RUNS} runs for instance {instance} finished at {datetime.now().strftime("%Y%m%d_%H%M%S")}')
        print(
            f'Best solution for instance {instance}: {min(solution_list, key=lambda x: x[0])}')
        print(
            f'Worst solution for instance {instance}: {max(solution_list or [], key=lambda x: x[0])}')

        if SAVE_RESULTS:
            os.makedirs(
                f'{BASE_RESULTS_FOLDER}/{results_folder}', exist_ok=True)
            save_results_on_file(f'{BASE_RESULTS_FOLDER}/{results_folder}', file_name, ant_type, cluster_type,
                                 solution_list, instance, 'csv', 'energy,arcs,distance,time(ms)')

if len(instances_with_errors) > 0:
    print(f'Instances with errors: {instances_with_errors}')
    os.makedirs(f'{BASE_RESULTS_FOLDER}/{results_folder}', exist_ok=True)
    error_file = open(
        f'{BASE_RESULTS_FOLDER}/{THIS_EXECUTION_FOLDER}/error_instances.txt', 'a')
    for instance, error in instances_with_errors:
        error_file.write(f'{instance}: {error}\n')
    error_file.close()


• INICIALIZANDO ALGORITMO •
------------------------------

• PARAMETROS DE LA INSTANCIA: 

    > Nodos y demandas: {0: 0, 1: 7, 2: 30, 3: 16, 4: 9, 5: 21, 6: 15, 7: 19, 8: 23, 9: 11, 10: 5, 11: 19, 12: 29, 13: 23, 14: 21, 15: 10, 16: 15, 17: 3, 18: 41, 19: 9, 20: 28, 21: 8, 22: 8, 23: 16, 24: 10, 25: 28, 26: 7, 27: 15, 28: 14, 29: 6, 30: 19, 31: 11, 32: 12, 33: 23, 34: 26, 35: 17, 36: 6, 37: 9, 38: 15, 39: 14, 40: 7, 41: 27, 42: 13, 43: 11, 44: 16, 45: 10, 46: 5, 47: 25, 48: 17, 49: 18, 50: 10}
    > Demanda total: 777
    > Capacidad vehiculo: 160
    > K-Optimo: 5
    > Tightnessratio: 0.97125

• INICIANDO CLUSTERIZACIÓN: 

    > CLUSTER TYPE: KMEANS
    > ITERACIÓN 1
        - Centroides actuales: [array([57, 58]), array([17, 33]), array([10, 17]), array([61, 33]), array([ 7, 38])]
        - Nodos sin asignar: [ 7 30], [23 11]
        - Mejor costo anterior: inf
        - Costo total actual: 9739.5597891085

    > ITERACIÓN 2
        - Centroides actuales: [array([37., 69.]), array

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:410: RuntimeWarning: divide by zero encountered in true_divide
  _distance_matrix = self.np.divide(1, self.distances_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:411: RuntimeWarning: divide by zero encountered in true_divide
  _energy_matrix = self.np.divide(1, self.energies_matrix)


        - Centroides actuales: [array([48.90909091, 58.36363636]), array([50.8, 35.6]), array([5., 6.]), array([38.5, 18.1]), array([19.2, 55.8])]
        - Nodos sin asignar: [38], [14]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 9938.838033288544

    > ITERACIÓN 39
        - Centroides actuales: [array([48.90909091, 58.36363636]), array([50.8, 35.6]), array([12.25, 21.  ]), array([59., 15.]), array([19.2, 55.8])]
        - Nodos sin asignar: [24 42], [28 11]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 8844.914022470824

    > ITERACIÓN 40
        - Centroides actuales: [array([48.36363636, 57.72727273]), array([46.6, 36.5]), array([ 7., 38.]), array([45.77777778, 18.77777778]), array([ 5., 64.])]
        - Nodos sin asignar: [40], [27]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 10771.024281503887

    > ITERACIÓN 41
        - Centroides actuales: [array([48.18181818, 52.27272

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:410: RuntimeWarning: divide by zero encountered in true_divide
  _distance_matrix = self.np.divide(1, self.distances_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:411: RuntimeWarning: divide by zero encountered in true_divide
  _energy_matrix = self.np.divide(1, self.energies_matrix)


        - Centroides actuales: [array([51.44444444, 58.55555556]), array([40.4, 40.7]), array([24.5, 18.5]), array([44.1, 23.5]), array([ 7., 38.])]
        - Nodos sin asignar: [ 6 42 25], [19 11  7]
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 9149.19296884423

    > ITERACIÓN 13
        - Centroides actuales: [array([27., 68.]), array([38.8, 42.6]), array([21.4, 22.6]), array([49. , 22.3]), array([ 5., 64.])]
        - Nodos sin asignar: [20], [8]
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 11671.139732542391

    > ITERACIÓN 14
        - Centroides actuales: [array([43.9, 63.2]), array([62., 42.]), array([20. , 19.7]), array([49. , 22.3]), array([16.55555556, 48.44444444])]
        - Nodos sin asignar: [39], [7]
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 10696.56634254118

    > ITERACIÓN 15
        - Centroides actuales: [array([41.16666667, 61.58333333]), array([47.77777778, 3

...

        - Nodos sin asignar: [32], [23]
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 9746.613717133847

    > ITERACIÓN 18
        - Centroides actuales: [array([44.9, 59.3]), array([54.63636364, 37.09090909]), array([15.88888889, 20.55555556]), array([46., 10.]), array([15.33333333, 54.        ])]
        - Nodos sin asignar: [35], [6]
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 9550.93543363689

    > ITERACIÓN 19
        - Centroides actuales: [array([63., 69.]), array([54.8, 40. ]), array([18.88888889, 22.33333333]), array([38.5, 18.3]), array([17.3, 50.5])]
        - Nodos sin asignar: [], []
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 10439.904257073627

    > ITERACIÓN 20
        - Centroides actuales: [array([47.3, 61. ]), array([48.09090909, 35.36363636]), array([18.88888889, 24.22222222]), array([41.2, 19.9]), array([16.8, 53.4])]
        - Nodos sin asignar: [24 25], [28  

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:410: RuntimeWarning: divide by zero encountered in true_divide
  _distance_matrix = self.np.divide(1, self.distances_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:411: RuntimeWarning: divide by zero encountered in true_divide
  _energy_matrix = self.np.divide(1, self.energies_matrix)


        - Centroides actuales: [array([51.6, 56.2]), array([53. , 31.5]), array([12.71428571, 25.28571429]), array([32.33333333, 23.83333333]), array([ 5., 64.])]
        - Nodos sin asignar: [32], [23]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 9209.872196275994

    > ITERACIÓN 37
        - Centroides actuales: [array([48.90909091, 58.36363636]), array([53.27272727, 32.09090909]), array([12.        , 27.71428571]), array([46., 10.]), array([18.5, 52.2])]
        - Nodos sin asignar: [18 39], [9 7]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 8682.729335656946

    > ITERACIÓN 38
        - Centroides actuales: [array([48.90909091, 58.36363636]), array([50.8, 35.6]), array([5., 6.]), array([38.5, 18.1]), array([19.2, 55.8])]
        - Nodos sin asignar: [38], [14]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 9938.838033288544

    > ITERACIÓN 39
        - Centroides actuales: [arr

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:410: RuntimeWarning: divide by zero encountered in true_divide
  _distance_matrix = self.np.divide(1, self.distances_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:411: RuntimeWarning: divide by zero encountered in true_divide
  _energy_matrix = self.np.divide(1, self.energies_matrix)


        - Centroides actuales: [array([40.54545455, 57.        ]), array([44.6, 42.5]), array([20. , 19.7]), array([50.7, 25. ]), array([17.        , 50.11111111])]
        - Nodos sin asignar: [12], [23]
        - Mejor costo anterior: 9131.213913563224
        - Costo total actual: 9205.552639738571

    > ITERACIÓN 8
        - Centroides actuales: [array([43.8, 61.5]), array([47.        , 39.45454545]), array([ 5., 25.]), array([50.11111111, 23.66666667]), array([15.66666667, 51.77777778])]
        - Nodos sin asignar: [11], [29]
        - Mejor costo anterior: 9131.213913563224
        - Costo total actual: 9274.450376400218

    > ITERACIÓN 9
        - Centroides actuales: [array([43.72727273, 58.54545455]), array([31., 32.]), array([12.25, 21.  ]), array([45.18181818, 21.36363636]), array([17.66666667, 51.11111111])]
        - Nodos sin asignar: [], []
        - Mejor costo anterior: 9131.213913563224
        - Costo total actual: 10115.12837053417

    > ITERACIÓN 10
        - C